In [1]:
%matplotlib 
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt4Agg


In [2]:
import sys
sys.path.append("/mnt/Data/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/")

In [126]:
import os
import numpy as np
import matplotlib.pyplot as pl
import cPickle as pkl
import colormaps as mycmps
import signale.tools as tools
import pandas as pd
import seaborn as sns
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import scipy.signal as scsig
import signale

In [100]:
def zScore(array):
    return(array - np.average(array))/ np.std(array)
def RippTrigeredPopRate(lfpPath,Tfolder,rippDB):
    rDB = pd.read_pickle(rippDB)
    tpaths = sorted([os.path.join(tFile[0],tFile[1]) for tFile in tools.locate('*.t',Tfolder)])
    lfp = pkl.load(open(lfpPath))
    bins = np.arange(np.floor(lfp.timeAxis[0]),np.floor(lfp.timeAxis[-1])+1,10)
    r = np.zeros(bins.size-1)
    binC = bins[:-1]+5
    for tFile in tpaths:
        t = signale.io.load_tFile(tFile)
        r += np.histogram(t.spike_times,bins)[0]
    r = zScore(r)
    rDB = pd.read_pickle(rippDB)
    lfpFile = lfpPath.split('/')[-1]
    session = lfpFile.split('_')[1]
    session = session.split('p')[0]+'p0'+session.split('p')[1]
    date = lfpFile.split('_')[0]
    relevantRipples = rDB[rDB.date==date]
    relevantRipples = relevantRipples[relevantRipples.session==session]
    Rcond = np.logical_and(relevantRipples.c ==1, relevantRipples.wc == 1)
    Gcond = np.logical_and(relevantRipples.c ==2, relevantRipples.wc == 2)
    tsR = []
    tsG = []
    for t_p in relevantRipples.t_peak[Rcond]:
        idx,tStamp = tools.findNearest(binC,t_p)
        tsR.append(r[idx-5:idx+5])
    for t_p in relevantRipples.t_peak[Gcond]:
        idx,tStamp = tools.findNearest(binC,t_p)
        tsG.append(r[idx-5:idx+5])
    return tsR,tsG

In [119]:
lfpPaths = ['/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-05_sleep1_csc4.lfp',
            '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-05_sleep2_csc4.lfp',
            '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-05_sleep3_csc4.lfp',
            '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-07_sleep1_csc4.lfp',
            '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-07_sleep2_csc4.lfp',
            '/mnt/Data/ephysdata/Gerbils/G5/data/2011-08-07_sleep3_csc4.lfp',
            '/mnt/Data/ephysdata/Gerbils/G6/2011-09-23/2011-09-23_sleep1_CSC4.lfp',
            '/mnt/Data/ephysdata/Gerbils/G6/2011-09-23/2011-09-23_sleep2_CSC4.lfp',
            '/mnt/Data/ephysdata/Gerbils/G6/2011-09-23/2011-09-23_sleep3_CSC4.lfp']
tFolders = ['/home/chenani/dataWork/ali/Gerbils/gerbil5/2011-08-05/sleep1/',
            '/home/chenani/dataWork/ali/Gerbils/gerbil5/2011-08-05/sleep2/',
            '/home/chenani/dataWork/ali/Gerbils/gerbil5/2011-08-05/sleep3/',
            '/home/chenani/dataWork/ali/Gerbils/gerbil5/2011-08-07/sleep1/',
            '/home/chenani/dataWork/ali/Gerbils/gerbil5/2011-08-07/sleep2/',
            '/home/chenani/dataWork/ali/Gerbils/gerbil5/2011-08-07/sleep3/',
            '/home/chenani/dataWork/ali/Gerbils/gerbil6/CA1/2011-09-23/sleep1',
            '/home/chenani/dataWork/ali/Gerbils/gerbil6/CA1/2011-09-23/sleep2',
            '/home/chenani/dataWork/ali/Gerbils/gerbil6/CA1/2011-09-23/sleep3',
           ]
DBpath = '/mnt/Data/ephysdata/Gerbils/RipplesTimeDB.pd'

In [120]:
rr = []
gg = []
for item in zip(lfpPaths,tFolders):
    r,g = RippTrigeredPopRate(item[0],item[1],DBpath)
    rr.extend(r)
    gg.extend(g)

In [128]:
sns.tsplot(rr)
sns.tsplot(gg,color='r')

In [127]:
pkl.dump(rr,open('/mnt/Data/ephysdata/Gerbils/popRateRipp.npArr','wb'),pkl.HIGHEST_PROTOCOL)
pkl.dump(gg,open('/mnt/Data/ephysdata/Gerbils/popRateFGB.npArr','wb'),pkl.HIGHEST_PROTOCOL)